In [1]:
%load_ext autoreload
%autoreload 2

# Fixed time series
> Use time-based splits rather than random splits

Context: the `neuron_33`, `neuron_46`, and `traffic` datasets were split randomly during the initial paper and rebuttal.
It would be better to use time-based splits, as the data is time series data.

I overwrote the `neuron_33` and `neuron_46` datasets in HuggingFace with transforms based on the first 80% of time 
points, so let's use those:

In [1]:
import manify
from manify.utils.dataloaders import load_hf
from manify.utils.benchmarks import benchmark

In [7]:
# Neuron 33

features, _, _, labels = load_hf("neuron_33")

# Take first 80% of samples for training and last 20% for testing
train_size = int(0.8 * len(features))
train_features, train_labels = features[:train_size], labels[:train_size]
test_features, test_labels = features[train_size:], labels[train_size:]
print(train_features.shape, train_labels.shape, test_features.shape, test_labels.shape)

# Manifold
pm = manify.ProductManifold(signature=[(1, 1)] * 10)

# Benchmark: Product RF Ambient RF Tangent RF k-Neighbors Ambient MLP κ-GCN
benchmark(
    X=None,
    y=None,
    X_train=train_features,
    y_train=train_labels,
    X_test=test_features,
    y_test=test_labels,
    pm=pm,
    models=["product_rf", "sklearn_rf", "tangent_rf", "knn", "ambient_mlp", "kappa_gcn"],
)

torch.Size([424, 20]) torch.Size([424]) torch.Size([106, 20]) torch.Size([106])


ambient_mlp:   0%|          | 0/4000 [00:00<?, ?it/s]

kappa_gcn:   0%|          | 0/4000 [00:00<?, ?it/s]

KeyboardInterrupt: 